In [1]:
import numpy as np
import pandas as pd

In [2]:
# Erdos-Renyi generator
def erdos_renyi(n):
    adj_mat = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            adj_mat[i,j] = np.random.randint(low=0, high=2)
    return adj_mat

# Dynamical process


In [3]:
n=5
A = erdos_renyi(n)
A

array([[1., 0., 0., 1., 0.],
       [1., 1., 1., 0., 0.],
       [1., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [4]:
x = np.random.randint(low=[0 for i in range(n)], high=[2 for i in range(n)])
x

array([1, 0, 1, 1, 0])

In [5]:
y = np.matmul(A,x)
y

array([2., 2., 2., 1., 1.])

In [6]:
z = np.array([ int(y[i] > 1) for i in range(n) ])
z

array([1, 1, 1, 0, 0])

In [7]:
x_test = np.array(x)
for t in range(10):
    y = np.matmul(A,x_test)
    x_test = y * (y==1)
    print(x_test)


[0. 0. 0. 1. 1.]
[1. 0. 1. 1. 1.]
[0. 0. 0. 1. 1.]
[1. 0. 1. 1. 1.]
[0. 0. 0. 1. 1.]
[1. 0. 1. 1. 1.]
[0. 0. 0. 1. 1.]
[1. 0. 1. 1. 1.]
[0. 0. 0. 1. 1.]
[1. 0. 1. 1. 1.]


In [8]:
x_test = np.array(x)
x_test

array([1, 0, 1, 1, 0])

In [9]:
y_test = np.matmul(A,x_test)
y_test * (y_test == 1)

array([0., 0., 0., 1., 1.])

In [10]:
import numpy as np
import pandas as pd

In [11]:
# Generate an adjacency matrix for a small network
n=3
A = np.zeros((n,n), dtype=int)
for i in range(n):
    for j in range(n):
        A[i,j] = np.random.choice([0,1])

print(A)

[[0 0 1]
 [0 1 0]
 [0 1 1]]


In [12]:
# Gives a random binary vector
def get_b(n):
    return np.array( [np.random.choice([complex("0"),complex("1")]) for i in range(n)] )

# In place, randomly selects an entry of 1 in b and replaces it with 1j.
def inject_tracker(b):
    support = [i for i in range(b.shape[0]) if b[i,] ]
    node = np.random.choice(support)
    b[node,] = complex("j")

# The collision function. The input is a vector y = Ax + b
def f(y):
    # We multiply each entry of y by the indicator that there is only one message at the corresponding node 
    return np.array( [ y[i,]*(y[i,].real + y[i,].imag == 1) for i in range(y.shape[0]) ] )




In [13]:
# Initialize external parameters
survival_times = dict()
alpha = 0
need_injection = True
T = 1000

# Initialize dynamical parameters (t=0)
x = get_b(n)

# This is the main loop
for t in range(0,T+1):
    # Get b_t
    b = get_b(n)

    # If it is possible and necessary to inject a tracker, do so, and update values accordingly
    if (np.sum(b) > 0) and need_injection:
        inject_tracker(b)
        need_injection = False
        alpha += 1
        survival_times[alpha] = 1
    
    # Calculate the next state
    x = f(np.matmul(A,x) + b)

    # Check if message alpha has survived, and if so, update accordingly
    if complex("j") in x:
        survival_times[alpha] += 1
    else:
        need_injection = True

In [14]:
data = pd.DataFrame.from_dict(survival_times, orient="index")
data = pd.DataFrame(data.value_counts())
data.sort_index(inplace=True)
data.columns=[1]
data.reset_index(inplace=True)
data.columns=["Survival_Time", "Count"]


data


,Survival_Time,Count
0,1,197
1,2,198
2,3,41
3,4,20
4,5,8
5,6,2
6,7,3
7,8,4
8,9,1
9,10,2


In [99]:
data

1    219
2    121
3     68
4     24
5     12
6     12
7      3
8      2
9      1
dtype: int64